# LSTM

- ## Preliminaries

- ### Imports

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,TimeDistributed
from keras.layers import LSTM,SimpleRNN
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import os, os.path
from os import listdir
from os.path import isfile, join
from unicodedata import normalize
import re

Using TensorFlow backend.


- ### Check GPU usage

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [3]:
get_available_gpus()

[u'/gpu:0']

----------

- ### Check and set Twitter's API

In [4]:
import twitter
api = twitter.Api(consumer_key='LyNVanTEQEOEGKfXAMeLv6AKG',
                    consumer_secret='0lJvhaaOP5cRZWm6rxwyBIAypd1P7eiDx9f74KBDlLrSldNuBQ',
                    access_token_key='855852332034265088-geTEVmA7xIsOD3WCZyfBNnqjRdS1MhW',
                    access_token_secret='kJMwMl67e3nYrqaGWzIizxzQpRZhtBfOnwPflO1fk3cOt')

In [5]:
print(api.VerifyCredentials())

{"created_at": "Sat Apr 22 18:34:31 +0000 2017", "default_profile": true, "description": "Learning how to be creative", "followers_count": 2, "friends_count": 3, "id": 855852332034265088, "lang": "en", "location": "Somewhere in the cloud", "name": "ArtistBot", "profile_background_color": "F5F8FA", "profile_banner_url": "https://pbs.twimg.com/profile_banners/855852332034265088/1492892354", "profile_image_url": "http://pbs.twimg.com/profile_images/855878764143804417/r55Z2Js5_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "TheTalkativeBot", "status": {"created_at": "Tue Apr 25 15:36:21 +0000 2017", "id": 856894659007844352, "id_str": "856894659007844352", "in_reply_to_screen_name": "dvp_tran", "in_reply_to_status_id": 856879788518236161, "in_reply_to_user_id": 747074580754403328, "lang": "de", "source": "<a href=\"http://www.google.com\" rel=\"nofollow\">TheScenarioBot</a>", "text": "@dvp_tran Automamte a

-------------------

# I. Learning from corpus

** 1. Load and convert data**

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

In [6]:
#Load and concatenate files:

DIR="../../LSTM/data/Gutenberg/ebooks-unzipped/French/"
all_files = [f for f in listdir(DIR) if isfile(join(DIR, f))]

#choose how many files to concatenate:
nb_files=7
if nb_files>len(all_files):
    nb_files=len(all_files)
    
    
out_path="french/data/"
if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(out_path+"input/"):
    os.makedirs(out_path+"input/")
    
with open(out_path+'input/french.txt', 'w') as outfile:
    for fname in all_files[0:nb_files]:
        with open(DIR+fname) as infile:
            i=0
            for line in infile:
                if i>=50:
                    outfile.write(line)
                i=i+1
        print ("Done concatenating file : %s" %fname)

Done concatenating file : 249.txt
Done concatenating file : 4740-8.txt
Done concatenating file : 799-0.txt
Done concatenating file : 4548-8.txt
Done concatenating file : 4791-8.txt
Done concatenating file : 803-8.txt
Done concatenating file : 2650-0.txt


In [7]:
#load file
file_name=out_path+'input/french.txt'
text = open(file_name).read()
text=normalize('NFKD',text.decode('latin1')).encode('ASCII', 'ignore')

In [8]:
text = re.sub("\n\n+" , "\n", text)

In [9]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print('total chars:',VOCAB_SIZE)

corpus length: 2508723
total chars: 89


**Warning:** The RNN takes in input numerical data hence the necessity to convert strings into numerical values.

In [10]:
#creating mapping between indexes and characters
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

We’re gonna use Keras to create and train our Network, so we must convert the data into this form: (number_of_sequences, length_of_sequence, number_of_features).
- nb of features = length of the char array
- length of sequence = batch size
- nb of sequence = len(data) divided by batch size.

**Warning : ** target sequence is setted by shifting the source/input sequence by one character with both having the same length.

In [11]:
%%time

SEQ_LENGTH=100
#Build three dimensional arrays
X = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #input
y = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #target

#Build sequences
for i in range(0, len(text)/SEQ_LENGTH):
    X_sequence = text[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_indices[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = text[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_indices[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

CPU times: user 2.02 s, sys: 192 ms, total: 2.22 s
Wall time: 2.22 s


** 2. Build the network**

In [12]:
HIDDEN_DIM= 500 #500
LAYER_NUM = 2


model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [13]:
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [14]:
# Generate some sample before training to know how bad it is!
bla = generate_text(model, 100, VOCAB_SIZE, indices_char)
#api.PostUpdate(status=bla[0:123])

[[[[@@@SSCCCCCBDD;;;GG3333q3333vvvvzzzzzzIIzMMnnnnnNNEEEiiimm4jjjE.SS!!!!!!!AAAAAAIkkk-SSSS!

**3. Train network**

In [15]:
def get_iternb(string):
    return re.findall(r'checkpoint_500_epoch_(.*).hdf5', string)[0]

In [ ]:
#batch size equals to seq length here
BATCH_SIZE=100
#len of desired output
GENERATE_LENGTH=140
DIR=out_path+"weights/weight_attempt_s03/"
flag=True

try:
    onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]
    iteration=[]
    for files in onlyfiles:
        iteration.append(int(get_iternb(files)))
    iteration=max(iteration)

    last_checkpoint=DIR+onlyfiles[0][0:21]+str(iteration)+'.hdf5'
except Exception as e:
    print(e)
    onlyfiles=[]
    if not os.path.exists(DIR):
        os.makedirs(DIR)
    nb_files=0

if nb_files>0:
    model.load_weights(last_checkpoint)
else:
    iteration=0
    
print("Starting at iteration : %s" %iteration)
while flag==True:
    print('\n')
    print('-'*20)
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=2, nb_epoch=1)
    iteration += 1
    bla=generate_text(model, GENERATE_LENGTH,VOCAB_SIZE, indices_char)
    if iteration % 10 == 0:
        print("\n\nIteration nb : %s" %iteration)
        #api.PostUpdate(status=bla[0:123])
        model.save_weights(DIR+'checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, iteration))
        #remove unecessary files:
        for files in onlyfiles:
            try:
                if files:
                    os.remove(DIR+files)
            except:
                pass
        onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]

    if iteration>=700:
        print("Stopping...")
        flag=False

Starting at iteration : 500


--------------------
Epoch 1/1
91s - loss: 0.2067
=

1.E.1.  The following sentence, with active links to, or other immediate
access to, the full Project Gutenberg-tm License must appear 

--------------------
Epoch 1/1
91s - loss: 0.2052
ble les attendrit. Elle reparut,
mais cette fois pour se suspendre dans l'air et quelque chose de cherche la
surface de la mer, le docteur

--------------------
Epoch 1/1
91s - loss: 0.2070
ge de terre argileuse et de sable, addition de ce briol ne
fut pas de part, reprit songer l'inconnu.  Un marbrant alime a
la meilleure rap

--------------------
Epoch 1/1
91s - loss: 0.2046
it de fuir les vainqueurs par son galop
du mont Ruse et comme je pense.

Qu'est-ce que cela veut savez, repliqua-t-il.

--Le Sueragen m

--------------------
Epoch 1/1
91s - loss: 0.2063
2., but its volunteers and employees are scattered
throughout numerous locations.  Its business office is located at
809 North 1500 West a

--------------------
Epo

**4. Generate text**

In [17]:
def save_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    if not os.path.exists(out_path+"generate/"):
        os.makedirs(out_path+"generate/")
    with open(out_path+"generate/output.txt","w") as f:
        f.write(('').join(y_char))
    return ('').join(y_char)


In [18]:
#seed with particular text:
def generate_text_seeded(model,seed,length, vocab_size, ix_to_char):
    # starting with random character
    # char_indices
    ix = [char_indices[x] for x in seed]
    y_char = [x for x in seed]
    X = np.zeros((1, length, vocab_size))
    for i in range(len(ix)) :
        X[0, i, :][ix[i]] = 1
        print(ix_to_char[ix[i]], end="")
    to_substract = len(ix)
    for i in range(length-to_substract):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [22]:
generate_text_seeded(model,normalize('NFKD',"le roi est mort ".decode('latin1')), 1000, VOCAB_SIZE, indices_char)

le roi est mort  auroi du tain entace euron fere tempe, comment malheureusement, et mille
vent se coucher, et qu'a gauche detourner son visage et
s'echauffe, furent un peu netant.

Tout d'un cost a construige. On saisit chacun en ces inscrites, et que le ciel veilleux que
son air devint fort d'habited.

J'etais perdu; il fallut un toluble confuse et n'avait plus de
crier les temps a la maison de baiser.

Ces paroles ne sont pas assez souffrants, sur le gravier de la terre,
s'ecriait:

Sur quelque haig!  Le gaz se courbe le premier mincre, que le coul, si cette decouverte, celui de
Hambourg, fit ma grand'plume et de plus penieux demander en ce moment.  Mon oncle
se levait a une voix rapide, ou tout absolu que je ne
pus en voir le courage de Cottard: qu'elle ne faisait que
les trois collA gees par ses excessions dont toute la
femme rose d'un brouh, comme vrai, depuis que Mme de Vallace oA1
inA. Mons de longues latins, nommAent aux calculs de ballon oA1 il avait rAussi 

u"le roi est mort auroi du tain entace euron fere tempe, comment malheureusement, et mille\r\nvent se coucher, et qu'a gauche detourner son visage et\r\ns'echauffe, furent un peu netant.\r\n\r\nTout d'un cost a construige. On saisit chacun en ces inscrites, et que le ciel veilleux que\r\nson air devint fort d'habited.\r\n\r\nJ'etais perdu; il fallut un toluble confuse et n'avait plus de\r\ncrier les temps a la maison de baiser.\r\n\r\nCes paroles ne sont pas assez souffrants, sur le gravier de la terre,\r\ns'ecriait:\r\n\r\nSur quelque haig!  Le gaz se courbe le premier mincre, que le coul, si cette decouverte, celui de\r\nHambourg, fit ma grand'plume et de plus penieux demander en ce moment.  Mon oncle\r\nse levait a une voix rapide, ou tout absolu que je ne\r\npus en voir le courage de Cottard: qu'elle ne faisait que\r\nles trois collA gees par ses excessions dont toute la\r\nfemme rose d'un brouh, comme vrai, depuis que Mme de Vallace oA1\r\ninA. Mons de longues latins, nommAent aux

In [ ]:
%%time
out = save_text(model, 1500, VOCAB_SIZE, indices_char)

our de la voir, celle-ci
Atait lourdA et magnifique.  Il apercevait qu'il fallait laigner
s'il ne l'a pas renouveler l'habitude.

Le mont PArous, que dA s que j'eus reconnu le plus de vrai, et A  aucune autre question a
l'excussionnalitA, la verdure de ses cheveux de plus de cinq pouces
au projectile.  Mais auprA s d'eux, tous les deux. Le honneme sentiment
rompu que son regard s'arrAata, un instant de la vivanon ligne de l'honorable EuropAenne: car
certaines fAates de la fonte de fonte peuvent Aatre retard et qui
malhent comme une pAtille angalotique, M. de Forcheville, Swann de
s'orateur, parmi les sapaquements relatifs aux accessoires A  l'amour du tableau ne vint A  l'antillitA, qui passait la
souscription profession d'access pour les enveloppes de son zenet.

Pendant une heure aprA s, elle s'interloguA intire quand il apercevait, aprA s avoir
rApondre avec calleurs, au coin de ses ouvrages cordiges de ses attachAtes. OA1
pour mon compte, meme souscription agrAmAe: un Acla frisque 